<h1>Project -  Stock Market Prediction</h1>


##Question 1: Pre-processing &  data exploration

In [315]:
import nltk
# nltk.download()
import string
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
import sklearn.datasets as sk
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [316]:
DataDirectoryPath= "C:\\Users\\nogahm\\PycharmProjects\\Project-Stock-Market-Prediction\\Data"

In [317]:
stopWords=(stopwords.words('english'))

In [318]:
# function that clean the text
def cleanText(text):
    text = str(text)
    if text.startswith("b\"") or text.startswith("b\'"):
        text = text[1:]
    text = text.replace("\"", "").replace("\'", "")
    # lower case
    text = text.lower()
    # tokenize
    filtered_sentence = word_tokenize(text)
    # # remove stop words
    filtered_sentence = [w for w in filtered_sentence if not w in stopWords]
    filtered_sentence = [w for w in filtered_sentence if len(w)>1]
    # stemming
    ps = PorterStemmer()
    for i in range(len(filtered_sentence)-1):
        try:
            filtered_sentence[i] = ps.stem(filtered_sentence[i])
        except Exception as inst:
            filtered_sentence[i]=filtered_sentence[i]
    # remove stop words after stem
    filtered_sentence = [w for w in filtered_sentence if not w in stopWords]
    filtered_sentence = [w for w in filtered_sentence if len(w)>1]
    text= " ".join(filtered_sentence)
    return text.strip()

In [319]:
# read the data into data frame
combinedDataDF = pd.read_csv( DataDirectoryPath+"\\Combined_News_DJIA.csv")

In [320]:
# clean the text (stopWords, tokenize, stemming)
headlines = combinedDataDF.columns[range(2, 27)]
combinedDataDF[headlines] = combinedDataDF[headlines].applymap(cleanText)

Class distribution:

In [321]:
classDist = []
for c in [0, 1]:
    temp=[c, combinedDataDF["Label"].value_counts()[c]]
    classDist.append(temp)
pd.DataFrame(classDist, columns=['Class', 'number of Samples'])

,Class,number of Samples
0,0,924
1,1,1065


Terms frequency:

In [322]:
# get 15 most common words for each class
termsFreqPerClass=[]
print ("Terms frequency - for each class, number of appearances of term:")
for currClass in [0, 1]:
    # aspect 1
    class_df = combinedDataDF.loc[combinedDataDF["Label"] == currClass]
    tempDF=pd.DataFrame()
    tempDF['text']=class_df[class_df.columns[2:]].apply(
        lambda x: " ".join(x.dropna()),
        axis=1
    )
    temp=[" ".join(set(w.split(" "))) for w in tempDF['text']] #remove duplicates
    temp=" ".join(temp)
    termFreq=pd.Series((temp).split()).value_counts()
    termsFreqDF=pd.DataFrame()
    termsFreqDF['Term']=termFreq.index
    termsFreqDF['Freq']=list(termFreq)
    setFreq=set(termsFreqDF['Freq'])
    # get top 12
    termFreq=termFreq[:12]
    zipedFreq=zip(termFreq.index,termFreq)
    zippedList=list(zipedFreq)
    termsFreqPerClass.append([currClass]+zippedList)
    # aspect 2
    print ("Class",currClass)
    word_frequency_per_class = pd.Series()
    for headline_column in combinedDataDF[headlines]:
        word_frequency_per_headline_column = pd.Series(" ".join(class_df[headline_column]).split()).value_counts()
        word_frequency_per_class = word_frequency_per_class.append(word_frequency_per_headline_column)
    word_frequency_per_class = word_frequency_per_class.groupby(by=word_frequency_per_class.index).sum().sort_values(ascending=False)[:12]
    print(word_frequency_per_class.to_frame('Word Frequency'))

print ("Terms frequency - for each class, number of records with term:")
pd.DataFrame(termsFreqPerClass,columns=['Class','term 1','term 2','term 3','term 4','term 5','term 6','term 7','term 8','term 9','term 10', 'term 11', 'term 12'])


Terms frequency - for each class, number of appearances of term:


('Class', 0)


        Word Frequency
say               1560
us                1365
new               1166
world             1153
kill              1145
year              1005
china              989
israel             970
govern             937
polic              849
peopl              848
russia             791


('Class', 1)


         Word Frequency
say                1809
us                 1544
kill               1375
world              1260
new                1254
china              1194
israel             1161
year               1142
govern             1071
polic               995
russia              939
protest             909
Terms frequency - for each class, number of records with term:


,Class,term 1,term 2,term 3,term 4,term 5,term 6,term 7,term 8,term 9,term 10,term 11,term 12
0,0,"(say, 748)","(us, 680)","(new, 650)","(world, 638)","(kill, 630)","(year, 583)","(govern, 572)","(china, 561)","(peopl, 549)","(report, 497)","(polic, 492)","(countri, 490)"
1,1,"(say, 836)","(us, 759)","(kill, 735)","(world, 706)","(new, 701)","(year, 676)","(china, 656)","(govern, 644)","(polic, 596)","(israel, 591)","(state, 591)","(peopl, 581)"


##Question 2: machine learning classifier

In [332]:
from sklearn.model_selection import train_test_split
# x = combined_news_djia_df[combined_news_djia_df.columns[2:4]]
x=combinedDataDF.drop(['Label'], axis=1)
y = combinedDataDF[["Label"]]

Split the data into train and test (30%-Test, 70%-Train)

In [333]:
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.3, random_state=0)

In [334]:
# combine all text column, and convert to list
xTrain = xTrain.apply(lambda x: " ".join(x), axis=1)
xTest = xTest.apply(lambda x: " ".join(x), axis=1)
xTrain=xTrain.tolist()
xTest=xTest.tolist()
yTrain=yTrain['Label'].tolist()
yTest=yTest['Label'].tolist()

In [335]:
import numpy as np
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron


The first model we will use is SVM (not Deep Learning, aka Keras)

In [327]:
results=[]
parameters = {'vect__max_df': (0.3,0.4,0.5,0.6,0.7,0.75, 1.0),'clf__alpha': (0.0001, 0.01,1.0)}
#TF-IDF & SVM
tfIdfVectorizer=TfidfVectorizer(sublinear_tf=True,stop_words='english')
ipeline = Pipeline([('vect', tf_idf_vectorizer), ('clf', SGDClassifier() )])
gs_clf = GridSearchCV(pipeline, parameters, n_jobs=1)
gs_clf = gs_clf.fit(xTrain, yTrain)
prediction = gs_clf.predict(xTest)
accuracy = metrics.accuracy_score(yTest, prediction)
params = gs_clf.best_params_
results.append(['TF-IDF' , 'SVM', params,accuracy])

In [328]:
print(results)
print 'AUC:', metrics.roc_auc_score(yTest, prediction)

[['TF-IDF', 'SVM', {'vect__max_df': 0.3, 'clf__alpha': 0.01}, 0.52931323283082077]]
AUC: 0.5


Run SVM with different pre-proccesing - remove five most common words (in order to check if accuracy/AUC improved)

In [336]:
removeWords={'say','us','new','kill','world'}
xTrainRemoveCommon=[w for w in xTrain if not w in removeWords]
xTestRemoveCommon=[w for w in xTest if not w in removeWords]
yTrainRemoveCommon=[w for w in yTrain if not w in removeWords]
yTestRemoveCommon=[w for w in yTest if not w in removeWords]

resultsCommon=[]
parametersCommon = {'vect__max_df': (0.3,0.4,0.5,0.6,0.7,0.75, 1.0),'clf__alpha': (0.0001, 0.01,1.0)}
#TF-IDF & SVM
tfIdfVectorizerCommon=TfidfVectorizer(sublinear_tf=True,stop_words='english')
ipelineCommon = Pipeline([('vect', tf_idf_vectorizer), ('clf', SGDClassifier() )])
gs_clfCommon = GridSearchCV(pipeline, parameters, n_jobs=1)
gs_clfCommon = gs_clf.fit(xTrain, yTrain)
predictionCommon = gs_clf.predict(xTest)
accuracyCommon = metrics.accuracy_score(yTest, prediction)
paramsCommon = gs_clf.best_params_
resultsCommon.append(['TF-IDF' , 'SVM', paramsCommon,accuracyCommon])

In [337]:
print(resultsCommon)
print 'AUC:', metrics.roc_auc_score(yTest, predictionCommon)

[['TF-IDF', 'SVM', {'vect__max_df': 0.3, 'clf__alpha': 0.01}, 0.52931323283082077]]
AUC: 0.5


The second model we will use is Neural Network (Deep Learning)

In [234]:
from keras.models import Sequential 
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

ImportError: No module named keras.models

In [1]:
#create the model
top_words=10000
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
xTrain2=tfIdfVectorizer.fit_transform(xTrain)
xTest2=tfIdfVectorizer.fit_transform(xTest)
model.fit(xTrain2, yTrain, validation_data=(xTest2, yTest), epochs=3, batch_size=128)

NameError: name 'Sequential' is not defined

get model accuracy

In [20]:
scores = model.evaluate(xTest2, yTest, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
y_pred_keras = model.predict(xTest2).ravel()
print 'AUC:', metrics.roc_auc_score(yTest, y_pred_keras)

##Question 3: Crawl updated data for 10 days

connect to reddit

In [329]:
import praw
import time
import datetime
import pandas as pd

reddit = praw.Reddit(client_id='pwzaClgaAR7lpw',
                     client_secret='90fJL78P0316g8IpW_0cGqfhmpM',
                     password='maya1234',
                     user_agent='danrave',
                     username='drave23')

get posts from the last month

In [330]:
new_posts=[]
for submission in reddit.subreddit('worldnews').top('month', limit=1000):
    # print(submission.title)
    new_posts.append(submission)

filtered the last 10 days

In [294]:
filtered = pd.DataFrame()
titles=[]
createDate=[]
start = 1549285503
end = 1550353401
for post in new_posts:
  if start < post.created_utc < end:  
    titles.append(post.title.encode('ascii', 'ignore'))
    createDate.append(datetime.datetime.fromtimestamp(post.created).strftime('%d/%m/%Y'))
filtered['Title']=titles
filtered['Date']=createDate

get 25 news from each day

In [296]:
newsDF={}
# print filtered
for index, row in filtered.iterrows():
    d= row['Date']
    if d not in newsDF:
         newsDF[d] = []
    if not (len(newsDF[d]) >= 25):
        newsDF[d].append(row['Title'])
newsDF = pd.DataFrame.from_dict(newsDF, orient='index').sort_index()
# print(newsDF)

Pre-process data from reddit

In [297]:
headlines_columns = newsDF.columns[range(0, 25)]
newsDF[headlines_columns] = newsDF[headlines_columns].applymap(cleanText)

Get label from Yahoo

In [298]:
from bs4 import BeautifulSoup
import urllib

In [299]:
yahooPage = urllib.urlopen("https://finance.yahoo.com/quote/%5EDJI/history?p=%5EDJI")
soup = BeautifulSoup(yahooPage, "html.parser")
djiaTable = pd.read_html(str(soup.find("table", class_="W(100%) M(0)")), header=0)[0]
djiaTable = djiaTable.head(10)

Calculate label by Yahoo information

In [300]:
labels=[]
dates=[]
for index, row in djiaTable.iterrows():
    open=row['Open']
    close=row['Close*']
    # not increase
    if(open>close):
        labels.append(0)
    # increase
    else:
        labels.append(1)
    dates.append(datetime.datetime.strptime(row['Date'], '%b %d, %Y').strftime('%d/%m/%Y'))
djiaLabeledDF=pd.DataFrame()
djiaLabeledDF['Date']=dates
djiaLabeledDF['Label']=labels

In [301]:
# join news with labels by dates
newsWithLabel = pd.merge(djiaLabeledDF[["Date", "Label"]], newsDF, left_on="Date", right_index=True)

Predict 10 days by SVM:

In [304]:
x=newsWithLabel.drop(['Label','Date'], axis=1)
y = newsWithLabel[["Label"]]
x=x.apply(lambda x: " ".join(x), axis=1).tolist()
prediction = gs_clf.predict(x)
print "Prediction using SVM:"
print(prediction)
print "Accuracy using SVM:"
print(metrics.accuracy_score(y, prediction))

Predict 10 days by RNN-LSTM:

In [307]:
predictionRNN=model.predict(x)
print "Prediction using RNN:"
print(predictionRNN)
print "Accuracy using RNN:"
print(metrics.accuracy_score(y, predictionRNN))